In [71]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup, Comment
import re
import pandas as pd
import os
import numpy as np


In [81]:
def check_xpath_exist(xpath):
    try:
        driver.find_element_by_xpath(xpath)
        return True
    except:
        return False
    
def scrape(links):
    data = pd.DataFrame(columns=['Topic', 'Body', 'in_post_url','Additional Info', 'facebook_url', 'Likes', 'Comments', 'Shares']) 
    currUrl = driver.current_url
    
    for link in links:
        driver.get(link)
    
        soup1 = BeautifulSoup(driver.page_source, 'lxml')
        
        body = soup1.find('div', {'class':'ads-info'}).text
        additional_info = soup1.find('ul', {'class':'list-custom'}).text
        

        topic = soup1.find(class_='entry-title').get_text()

        newUrl = soup1.find('iframe', {'allow': 'encrypted-media'})['src']
        driver.get(newUrl)

        soup2=BeautifulSoup(driver.page_source, 'lxml')
        
        like = soup2.find('span', {'class':'embeddedLikeButton'}).get_text()
        comment = soup2.find('div', {'title':'Comment'}).get_text()
        share = soup2.find('div', {'title':'Share'}).get_text()
        
        try:
            in_post_link = soup1.find('div', {'class':'ads-info'}).find('a')['href']
        except:
            in_post_link = ""

        data = data.append({'Topic':topic, 'Body': body, 'in_post_url': in_post_link, 'Additional Info': additional_info, 'facebook_url':link, 'Likes':like, 'Comments':comment, 'Shares':share}, ignore_index=True)  
    
    driver.get(currUrl)
    return data

In [73]:
driver = webdriver.Remote(command_executor='http://127.0.0.1:9515', desired_capabilities=DesiredCapabilities.CHROME)

driver.get("https://adespresso.com/ads-examples/")

In [82]:
try:
    driver.find_element_by_name('firstname').send_keys("Daniel")
    driver.find_element_by_name('email').send_keys("daniel@gmail.com")
    driver.find_element_by_xpath("//input[@value='Count me In!']").click()
except:
    pass

# search = input("Search For:")
# driver.find_element_by_id('ads-search').send_keys(search)
# driver.find_element_by_id('ads-search').send_keys(Keys.ENTER)

load_older = "//a[contains(text(),'Older Ads')]"


page_num = 0

while check_xpath_exist(load_older):
    try:
        print("Scraping page {}".format(str(page_num)))
        initialSoup=BeautifulSoup(driver.page_source, 'lxml')

        links = initialSoup.find_all('li', {'class': 'sf-item'})
        links = list(set([link.find('a')['href'] for link in links]))

        soup=BeautifulSoup(driver.page_source, 'lxml')
        df = scrape(links)

        if page_num == 0:
            df.to_csv('data.csv', index=False)
        else:
            df.to_csv('data.csv', mode='a', header=False, index=False)

        page_num = page_num + 1
        driver.find_element_by_xpath(load_older).click()
    except Exception as e:
        print("Got an error: {}".format(str(e)))